In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score

# seed
import os
seed = 777
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
x_train = train.drop(['id', 'digit', 'letter'], axis=1).values
x_train = x_train.reshape(-1, 28, 28, 1)
x_train = x_train/255

y_data = train['digit']
y_train = np.zeros((len(y_data), len(y_data.unique())))
for i, digit in enumerate(y_data):
    y_train[i, digit] = 1

print(x_train.shape)
print(y_train.shape)

(2048, 28, 28, 1)
(2048, 10)


In [4]:
def create_model(con=3, drop=0.2, optimizer='adam', padding='valid'):
    model = Sequential()
    model.add(Conv2D(64, (con,con), input_shape=(28,28,1), activation='relu', padding=padding, name='conv1'))
    model.add(Dropout(drop))
    model.add(MaxPooling2D((2,2), name='pool1'))
    model.add(Conv2D(64, (2,2), activation='relu', padding=padding, name='conv2'))
    model.add(Dropout(drop))
    model.add(MaxPooling2D((2,2), name='pool2'))
    model.add(Conv2D(128, (2,2), activation='relu', padding=padding, name='conv3'))
    model.add(Dropout(drop))
    model.add(MaxPooling2D((2,2), name='pool3'))
    model.add(Flatten(name='flat1'))
    model.add(Dense(500, activation='relu', name='hidden1'))
    model.add(Dropout(drop))
    model.add(Dense(100, activation='relu', name='hidden2'))
    model.add(Dropout(drop))
    model.add(Dense(50, activation='relu', name='hidden3'))
    model.add(Dense(10, activation='softmax', name='output'))

    model.compile(optimizer = optimizer, metrics = ['accuracy'], 
                  loss = 'categorical_crossentropy')
    
    return model

In [5]:
model_struct = create_model(3, 0.3, 'adam', 'same')
model_struct.summary()
#plot_model(model_struct, show_shapes=True, to_file='struct.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 28, 28, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 64)        0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 14, 14, 64)        16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 7, 7, 64)          0         
_________________________________________________________________
conv3 (Conv2D)               (None, 7, 7, 128)         32896     
__________

In [6]:
def hyperparameters():
    con = [2, 3, 4]
    dropout = np.linspace(0.1, 0.5, 5).tolist()
    batches = [32, 64, 128]
    optimizers = ['adam', 'adadelta', 'rmsprop']
    padding = ['valid', 'same']
    return {'con':con, 'drop':dropout, 'batch_size':batches, 
            'optimizer':optimizers, 'padding':padding}

hyperparameters = hyperparameters()

In [9]:
model = KerasClassifier(build_fn = create_model, verbose=2)
search = RandomizedSearchCV(model, hyperparameters, cv=9, refit=True)#, scoring = make_scorer(accuracy_score))

In [10]:
best_model = search.fit(x_train, y_train, epochs=50)

Epoch 1/50
 - 2s - loss: 2.2948 - acc: 0.1258
Epoch 2/50
 - 1s - loss: 2.1111 - acc: 0.2308
Epoch 3/50
 - 1s - loss: 1.7907 - acc: 0.3725
Epoch 4/50
 - 1s - loss: 1.5021 - acc: 0.5022
Epoch 5/50
 - 1s - loss: 1.3092 - acc: 0.5797
Epoch 6/50
 - 1s - loss: 1.1956 - acc: 0.6044
Epoch 7/50
 - 1s - loss: 1.0940 - acc: 0.6401
Epoch 8/50
 - 1s - loss: 0.9559 - acc: 0.6846
Epoch 9/50
 - 1s - loss: 0.9250 - acc: 0.6868
Epoch 10/50
 - 1s - loss: 0.8262 - acc: 0.7203
Epoch 11/50
 - 1s - loss: 0.7526 - acc: 0.7522
Epoch 12/50
 - 1s - loss: 0.7195 - acc: 0.7560
Epoch 13/50
 - 1s - loss: 0.6624 - acc: 0.7780
Epoch 14/50
 - 1s - loss: 0.6290 - acc: 0.7852
Epoch 15/50
 - 1s - loss: 0.5688 - acc: 0.8088
Epoch 16/50
 - 1s - loss: 0.5154 - acc: 0.8203
Epoch 17/50
 - 1s - loss: 0.4758 - acc: 0.8363
Epoch 18/50
 - 1s - loss: 0.4644 - acc: 0.8423
Epoch 19/50
 - 1s - loss: 0.4351 - acc: 0.8505
Epoch 20/50
 - 1s - loss: 0.3895 - acc: 0.8643
Epoch 21/50
 - 1s - loss: 0.3622 - acc: 0.8731
Epoch 22/50
 - 1s - lo

 - 1s - loss: 0.2617 - acc: 0.9093
Epoch 27/50
 - 1s - loss: 0.2530 - acc: 0.9082
Epoch 28/50
 - 1s - loss: 0.2390 - acc: 0.9121
Epoch 29/50
 - 1s - loss: 0.2015 - acc: 0.9302
Epoch 30/50
 - 1s - loss: 0.2000 - acc: 0.9313
Epoch 31/50
 - 1s - loss: 0.1849 - acc: 0.9385
Epoch 32/50
 - 1s - loss: 0.1821 - acc: 0.9297
Epoch 33/50
 - 1s - loss: 0.1744 - acc: 0.9401
Epoch 34/50
 - 1s - loss: 0.1759 - acc: 0.9396
Epoch 35/50
 - 1s - loss: 0.1580 - acc: 0.9440
Epoch 36/50
 - 1s - loss: 0.1729 - acc: 0.9456
Epoch 37/50
 - 1s - loss: 0.1499 - acc: 0.9440
Epoch 38/50
 - 1s - loss: 0.1357 - acc: 0.9522
Epoch 39/50
 - 1s - loss: 0.1193 - acc: 0.9659
Epoch 40/50
 - 1s - loss: 0.1224 - acc: 0.9621
Epoch 41/50
 - 1s - loss: 0.1222 - acc: 0.9577
Epoch 42/50
 - 1s - loss: 0.1056 - acc: 0.9610
Epoch 43/50
 - 1s - loss: 0.1211 - acc: 0.9566
Epoch 44/50
 - 1s - loss: 0.1250 - acc: 0.9588
Epoch 45/50
 - 1s - loss: 0.1036 - acc: 0.9654
Epoch 46/50
 - 1s - loss: 0.1044 - acc: 0.9621
Epoch 47/50
 - 1s - loss:

 - 2s - loss: 2.2880 - acc: 0.1230
Epoch 2/50
 - 1s - loss: 2.1459 - acc: 0.1977
Epoch 3/50
 - 1s - loss: 1.9121 - acc: 0.3119
Epoch 4/50
 - 1s - loss: 1.6378 - acc: 0.4382
Epoch 5/50
 - 1s - loss: 1.4057 - acc: 0.5228
Epoch 6/50
 - 1s - loss: 1.2289 - acc: 0.5837
Epoch 7/50
 - 1s - loss: 1.1027 - acc: 0.6310
Epoch 8/50
 - 1s - loss: 0.9941 - acc: 0.6787
Epoch 9/50
 - 1s - loss: 0.9296 - acc: 0.6804
Epoch 10/50
 - 1s - loss: 0.8638 - acc: 0.7117
Epoch 11/50
 - 1s - loss: 0.7910 - acc: 0.7359
Epoch 12/50
 - 1s - loss: 0.7180 - acc: 0.7496
Epoch 13/50
 - 1s - loss: 0.6957 - acc: 0.7666
Epoch 14/50
 - 1s - loss: 0.6407 - acc: 0.7858
Epoch 15/50
 - 1s - loss: 0.5864 - acc: 0.7941
Epoch 16/50
 - 1s - loss: 0.5499 - acc: 0.8138
Epoch 17/50
 - 1s - loss: 0.5012 - acc: 0.8331
Epoch 18/50
 - 1s - loss: 0.4516 - acc: 0.8380
Epoch 19/50
 - 1s - loss: 0.4421 - acc: 0.8495
Epoch 20/50
 - 1s - loss: 0.4249 - acc: 0.8523
Epoch 21/50
 - 1s - loss: 0.3869 - acc: 0.8605
Epoch 22/50
 - 1s - loss: 0.3729 

Epoch 27/50
 - 0s - loss: 0.2721 - acc: 0.9049
Epoch 28/50
 - 0s - loss: 0.2621 - acc: 0.9121
Epoch 29/50
 - 0s - loss: 0.2397 - acc: 0.9187
Epoch 30/50
 - 0s - loss: 0.2024 - acc: 0.9291
Epoch 31/50
 - 0s - loss: 0.1969 - acc: 0.9363
Epoch 32/50
 - 0s - loss: 0.1752 - acc: 0.9434
Epoch 33/50
 - 0s - loss: 0.1761 - acc: 0.9396
Epoch 34/50
 - 0s - loss: 0.1719 - acc: 0.9445
Epoch 35/50
 - 0s - loss: 0.1332 - acc: 0.9549
Epoch 36/50
 - 0s - loss: 0.1473 - acc: 0.9522
Epoch 37/50
 - 0s - loss: 0.1241 - acc: 0.9571
Epoch 38/50
 - 0s - loss: 0.1112 - acc: 0.9632
Epoch 39/50
 - 0s - loss: 0.1208 - acc: 0.9626
Epoch 40/50
 - 0s - loss: 0.1181 - acc: 0.9588
Epoch 41/50
 - 0s - loss: 0.1143 - acc: 0.9610
Epoch 42/50
 - 0s - loss: 0.0979 - acc: 0.9709
Epoch 43/50
 - 0s - loss: 0.0901 - acc: 0.9698
Epoch 44/50
 - 0s - loss: 0.0744 - acc: 0.9753
Epoch 45/50
 - 0s - loss: 0.0754 - acc: 0.9764
Epoch 46/50
 - 0s - loss: 0.0959 - acc: 0.9681
Epoch 47/50
 - 0s - loss: 0.0633 - acc: 0.9786
Epoch 48/50
 

Epoch 2/50
 - 0s - loss: 2.1568 - acc: 0.2048
Epoch 3/50
 - 0s - loss: 1.9191 - acc: 0.3103
Epoch 4/50
 - 0s - loss: 1.7051 - acc: 0.4201
Epoch 5/50
 - 0s - loss: 1.5519 - acc: 0.4640
Epoch 6/50
 - 0s - loss: 1.4040 - acc: 0.5113
Epoch 7/50
 - 0s - loss: 1.2442 - acc: 0.5881
Epoch 8/50
 - 0s - loss: 1.0965 - acc: 0.6277
Epoch 9/50
 - 0s - loss: 1.0213 - acc: 0.6551
Epoch 10/50
 - 0s - loss: 0.8967 - acc: 0.7013
Epoch 11/50
 - 0s - loss: 0.8560 - acc: 0.7133
Epoch 12/50
 - 0s - loss: 0.7911 - acc: 0.7315
Epoch 13/50
 - 0s - loss: 0.7542 - acc: 0.7446
Epoch 14/50
 - 0s - loss: 0.6730 - acc: 0.7699
Epoch 15/50
 - 0s - loss: 0.6376 - acc: 0.7781
Epoch 16/50
 - 0s - loss: 0.5988 - acc: 0.7941
Epoch 17/50
 - 0s - loss: 0.5568 - acc: 0.8094
Epoch 18/50
 - 0s - loss: 0.5082 - acc: 0.8265
Epoch 19/50
 - 0s - loss: 0.4638 - acc: 0.8435
Epoch 20/50
 - 0s - loss: 0.4576 - acc: 0.8528
Epoch 21/50
 - 0s - loss: 0.3988 - acc: 0.8638
Epoch 22/50
 - 0s - loss: 0.3622 - acc: 0.8627
Epoch 23/50
 - 0s - l

...

 - 1s - loss: 0.1788 - acc: 0.9368
Epoch 30/50
 - 1s - loss: 0.1576 - acc: 0.9440
Epoch 31/50
 - 1s - loss: 0.1318 - acc: 0.9544
Epoch 32/50
 - 1s - loss: 0.1203 - acc: 0.9643
Epoch 33/50
 - 1s - loss: 0.1434 - acc: 0.9527
Epoch 34/50
 - 1s - loss: 0.1627 - acc: 0.9429
Epoch 35/50
 - 1s - loss: 0.1044 - acc: 0.9632
Epoch 36/50
 - 1s - loss: 0.0883 - acc: 0.9698
Epoch 37/50
 - 1s - loss: 0.0768 - acc: 0.9780
Epoch 38/50
 - 1s - loss: 0.0975 - acc: 0.9720
Epoch 39/50
 - 1s - loss: 0.0658 - acc: 0.9764
Epoch 40/50
 - 1s - loss: 0.0781 - acc: 0.9775
Epoch 41/50
 - 1s - loss: 0.0585 - acc: 0.9824
Epoch 42/50
 - 1s - loss: 0.0762 - acc: 0.9725
Epoch 43/50
 - 1s - loss: 0.0754 - acc: 0.9780
Epoch 44/50
 - 1s - loss: 0.0752 - acc: 0.9725
Epoch 45/50
 - 1s - loss: 0.0594 - acc: 0.9841
Epoch 46/50
 - 1s - loss: 0.0549 - acc: 0.9824
Epoch 47/50
 - 1s - loss: 0.0516 - acc: 0.9819
Epoch 48/50
 - 1s - loss: 0.0544 - acc: 0.9824
Epoch 49/50
 - 1s - loss: 0.0469 - acc: 0.9852
Epoch 50/50
 - 1s - loss:

 - 1s - loss: 1.8527 - acc: 0.3454
Epoch 5/50
 - 1s - loss: 1.6858 - acc: 0.4108
Epoch 6/50
 - 1s - loss: 1.5002 - acc: 0.4926
Epoch 7/50
 - 1s - loss: 1.3343 - acc: 0.5442
Epoch 8/50
 - 1s - loss: 1.1972 - acc: 0.6008
Epoch 9/50
 - 1s - loss: 1.0816 - acc: 0.6359
Epoch 10/50
 - 1s - loss: 1.0045 - acc: 0.6535
Epoch 11/50
 - 1s - loss: 0.9388 - acc: 0.6815
Epoch 12/50
 - 1s - loss: 0.8316 - acc: 0.7084
Epoch 13/50
 - 1s - loss: 0.7606 - acc: 0.7381
Epoch 14/50
 - 1s - loss: 0.7022 - acc: 0.7617
Epoch 15/50
 - 1s - loss: 0.6541 - acc: 0.7787
Epoch 16/50
 - 1s - loss: 0.5833 - acc: 0.8116
Epoch 17/50
 - 1s - loss: 0.5495 - acc: 0.8100
Epoch 18/50
 - 1s - loss: 0.5190 - acc: 0.8243
Epoch 19/50
 - 1s - loss: 0.4525 - acc: 0.8468
Epoch 20/50
 - 1s - loss: 0.4172 - acc: 0.8567
Epoch 21/50
 - 1s - loss: 0.3732 - acc: 0.8677
Epoch 22/50
 - 1s - loss: 0.3535 - acc: 0.8764
Epoch 23/50
 - 1s - loss: 0.3263 - acc: 0.8836
Epoch 24/50
 - 1s - loss: 0.2886 - acc: 0.9017
Epoch 25/50
 - 1s - loss: 0.25

 - 1s - loss: 0.1986 - acc: 0.9248
Epoch 30/50
 - 1s - loss: 0.1990 - acc: 0.9319
Epoch 31/50
 - 1s - loss: 0.1445 - acc: 0.9550
Epoch 32/50
 - 1s - loss: 0.1416 - acc: 0.9517
Epoch 33/50
 - 1s - loss: 0.1562 - acc: 0.9495
Epoch 34/50
 - 1s - loss: 0.1196 - acc: 0.9599
Epoch 35/50
 - 1s - loss: 0.1112 - acc: 0.9616
Epoch 36/50
 - 1s - loss: 0.0955 - acc: 0.9725
Epoch 37/50
 - 1s - loss: 0.0822 - acc: 0.9736
Epoch 38/50
 - 1s - loss: 0.1069 - acc: 0.9610
Epoch 39/50
 - 1s - loss: 0.0943 - acc: 0.9671
Epoch 40/50
 - 1s - loss: 0.0884 - acc: 0.9731
Epoch 41/50
 - 1s - loss: 0.0917 - acc: 0.9681
Epoch 42/50
 - 1s - loss: 0.0646 - acc: 0.9775
Epoch 43/50
 - 1s - loss: 0.0629 - acc: 0.9786
Epoch 44/50
 - 1s - loss: 0.0586 - acc: 0.9786
Epoch 45/50
 - 1s - loss: 0.0742 - acc: 0.9747
Epoch 46/50
 - 1s - loss: 0.0758 - acc: 0.9720
Epoch 47/50
 - 1s - loss: 0.0624 - acc: 0.9841
Epoch 48/50
 - 1s - loss: 0.0628 - acc: 0.9830
Epoch 49/50
 - 1s - loss: 0.0393 - acc: 0.9896
Epoch 50/50
 - 1s - loss:

 - 0s - loss: 2.2351 - acc: 0.1747
Epoch 5/50
 - 0s - loss: 2.1566 - acc: 0.1984
Epoch 6/50
 - 0s - loss: 2.0487 - acc: 0.2456
Epoch 7/50
 - 0s - loss: 2.0145 - acc: 0.2714
Epoch 8/50
 - 0s - loss: 1.8880 - acc: 0.3225
Epoch 9/50
 - 0s - loss: 1.8312 - acc: 0.3434
Epoch 10/50
 - 0s - loss: 1.7899 - acc: 0.3841
Epoch 11/50
 - 0s - loss: 1.7924 - acc: 0.3709
Epoch 12/50
 - 0s - loss: 1.7054 - acc: 0.4082
Epoch 13/50
 - 0s - loss: 1.6506 - acc: 0.4286
Epoch 14/50
 - 0s - loss: 1.5768 - acc: 0.4698
Epoch 15/50
 - 0s - loss: 1.5815 - acc: 0.4648
Epoch 16/50
 - 0s - loss: 1.4320 - acc: 0.5049
Epoch 17/50
 - 0s - loss: 1.4053 - acc: 0.5363
Epoch 18/50
 - 0s - loss: 1.2986 - acc: 0.5659
Epoch 19/50
 - 0s - loss: 1.3321 - acc: 0.5473
Epoch 20/50
 - 0s - loss: 1.1432 - acc: 0.6242
Epoch 21/50
 - 0s - loss: 1.1089 - acc: 0.6269
Epoch 22/50
 - 0s - loss: 1.1049 - acc: 0.6214
Epoch 23/50
 - 0s - loss: 1.0424 - acc: 0.6462
Epoch 24/50
 - 0s - loss: 0.9814 - acc: 0.6681
Epoch 25/50
 - 0s - loss: 0.97

 - 0s - loss: 0.9091 - acc: 0.6859
Epoch 30/50
 - 0s - loss: 0.8269 - acc: 0.7155
Epoch 31/50
 - 0s - loss: 0.8360 - acc: 0.7232
Epoch 32/50
 - 0s - loss: 0.7671 - acc: 0.7326
Epoch 33/50
 - 0s - loss: 0.7495 - acc: 0.7512
Epoch 34/50
 - 0s - loss: 0.7455 - acc: 0.7468
Epoch 35/50
 - 0s - loss: 0.7614 - acc: 0.7403
Epoch 36/50
 - 0s - loss: 0.7183 - acc: 0.7474
Epoch 37/50
 - 0s - loss: 0.6999 - acc: 0.7584
Epoch 38/50
 - 0s - loss: 0.6762 - acc: 0.7672
Epoch 39/50
 - 0s - loss: 0.6070 - acc: 0.7886
Epoch 40/50
 - 0s - loss: 0.6128 - acc: 0.7836
Epoch 41/50
 - 0s - loss: 0.6573 - acc: 0.7694
Epoch 42/50
 - 0s - loss: 0.5752 - acc: 0.8122
Epoch 43/50
 - 0s - loss: 0.5317 - acc: 0.8193
Epoch 44/50
 - 0s - loss: 0.5142 - acc: 0.8243
Epoch 45/50
 - 0s - loss: 0.5673 - acc: 0.8072
Epoch 46/50
 - 0s - loss: 0.5217 - acc: 0.8188
Epoch 47/50
 - 0s - loss: 0.5242 - acc: 0.8089
Epoch 48/50
 - 0s - loss: 0.5312 - acc: 0.8188
Epoch 49/50
 - 0s - loss: 0.5079 - acc: 0.8171
Epoch 50/50
 - 0s - loss:

In [11]:
print(search.best_params_)

{'padding': 'same', 'optimizer': 'adadelta', 'drop': 0.2, 'con': 2, 'batch_size': 64}


In [12]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255
x_test.shape

(20480, 28, 28, 1)

In [14]:
predict = best_model.predict(x_test)

In [18]:
len(predict)

20480

In [19]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = predict

In [20]:
submission.to_csv('data/submission1.csv', index=False)